In [16]:
import sys, os, math, tempfile, datetime, time, copy, re
import numpy as np
import pandas as pd
from allennlp.commands.elmo import ElmoEmbedder
from collections import Counter, defaultdict
from keras.layers import Input, Dense
from keras.models import Model 
from keras.models import model_from_json
from math import log
from pathlib import Path
import pickle
from scipy.stats import percentileofscore
from sklearn.preprocessing import LabelEncoder
import argparse
import matplotlib.pyplot as plt
import torch
import multiprocessing.pool

In [17]:
dict_E3_degrons = {'CBL': ['CBL APS motif', 'CBL MET motif', 'CBL PTK motif'],
                    'CDC20': ['APC D box', 'APC ABBA/Cdc20','APC ABBA motif','APC KEN box','APC TPR motif'],
                    'COP1': ['COP1 motif'], 'DTL': ['DTL PIP motif 1', 'DTL PIP motif 2'],
                    'CUL3': ['KLHL3/KLH2 motif', 'KLHL17 motif'], 'KLHL3': ['KLHL3/KLH2 motif'],
                    'KLHL2': ['KLHL3/KLH2 motif'], 'KEAP1': ['KEAP1 motif 1', 'KEAP1 motif 2'],
                    'KLHL17': ['KLHL17 motif'], 'MDM2': ['MDM2 motif'], 'VHL': ['VHL motif'],
                    'FBXW7': ['FBXW7 motif 1', 'FBXW7 motif 2'], 'SKP2': ['SKP2 Fbox motif'],
                    'BTRC': ['BTRCP motif'], 'SIAH2': ['SIAH motif'], 'SIAH1': ['SIAH motif'],
                    'SPOP': ['SPOP motif'], 'FBXL2': ['FBXL2 motif'], 'FBXO31': ['FBX031 motif'],
                    'NEDD4': ['ITCH motif'], 'ITCH': ['ITCH motif']}

covariates =["ASA_SCORE","DSS_SCORE","FCONS_SCORE","COIL","ANCHOR_SCORE","nflanking_ptms","nflanking_ub_lysines","HELIX","STRAND","RIG_SCORE","Domain_pfam"]
order = ["Entry","Hit","DEGRON","E3","START","END","Prob_DEGRON","ID_in_UniProt"]

classifier_XGBoost  = "./models/classifier_XGBoost.pickle"
uid_list = np.load('./supporting_file/uniprot_list.npy', allow_pickle = True)

path_output_classes = ("./supporting_file/elm_class_202309.txt")
df_dregon_types = pd.read_csv(path_output_classes,sep="\t")
d_motifs = {}
for index,row in df_dregon_types.iterrows():
    d_motifs[row["Motif"]]=re.compile(row["Regex"])

In [4]:
class Elmo_embedder():
    def __init__(self, model_dir="./models/uniref50_v2", weights="weights.hdf5",
                 options="options.json", threads=100): 
        if threads == 100:
            torch.set_num_threads(multiprocessing.cpu_count() // 2)
        else:
            torch.set_num_threads(threads)

        self.model_dir = Path(model_dir)
        self.weights = self.model_dir / weights
        self.options = self.model_dir / options
        self.seqvec = ElmoEmbedder(self.options, self.weights, cuda_device=-1)

    def elmo_embedding(self, x, start=None, stop=None):
        assert start is None and stop is None, "deprecated to use start stop, please trim seqs beforehand"

        if type(x[0]) == str:
            x = np.array([list(i.upper()) for i in x])
        embedding = self.seqvec.embed_sentences(x)
        X_parsed = []
        for i in embedding:
            X_parsed.append(i.mean(axis=0))
        return X_parsed

In [5]:
def read_fasta(fasta_file):
    try:
        fp = open(fasta_file)
    except IOError:
        exit()
    else:
        fp = open(fasta_file)
        lines = fp.readlines()
        fasta_dict = {} 
        gene_id = ""
        for line in lines:
            if line[0] == '>':
                if gene_id != "":
                    fasta_dict[gene_id] = seq
                seq = ""
                gene_id = line.strip()[1:]
            else:
                seq += line.strip()        
        fasta_dict[gene_id] = seq       
    return fasta_dict  

def AA_encoding(seq_extended):
    amino = "ABCDEFGHIJKLMNOPQRSTUVWXYZ-"
    encoder = LabelEncoder()
    encoder.fit(list(amino))
    seq_transformed = np.array(
        list(map(encoder.transform, np.array([list(i.upper()) for i in seq_extended]))))   
    return seq_transformed[0]

In [6]:
def import_model():
    json_f = open("./models/degron_DL.json", 'r')
    loaded_model_json = json_f.read()
    json_f.close()
    loaded_model = model_from_json(loaded_model_json)
    loaded_model.load_weights('./models/degron_DL.h5')
    return loaded_model

### Predicting E3 ligase targeted degrons using deep learning

In [8]:
def pred_and_write_result_data(fasta_file, out_file, degron_motif):
    
    ID_sequences = read_fasta(fasta_file)
    l_results=[]
    uid_degron = list(ID_sequences.keys())

    for E3_query in degron_motif:
        degron_motif = dict_E3_degrons[E3_query]
        
        for uid in uid_degron:
            seq_local = ID_sequences[uid]
            for motif in degron_motif:
                for m in re.finditer(d_motifs[motif], seq_local):
                    l_results.append([uid,E3_query,m.group(),motif,m.start()+1,m.end()])
                
    df_matches_seq = pd.DataFrame(l_results,columns=["Entry","E3","Hit","DEGRON","START","END"])   

    all_data = pd.DataFrame()
    
    if len(df_matches_seq) > 0:
        print('Deep learning models are making predictions...')
        model_DL = import_model()
        elmo_embedder = Elmo_embedder(threads=60)
        shift, slicesize = 14, 29

        d_nouid_degron_pos = {}
        for index, row in df_matches_seq.iterrows():
            if row["Entry"] not in d_nouid_degron_pos.keys():
                d_nouid_degron_pos[row["Entry"]] = ["_".join([str(row["START"]), str(row["END"]), row["Hit"], row["DEGRON"],row["E3"]])]
            else:
                d_nouid_degron_pos[row["Entry"]].append("_".join([str(row["START"]), str(row["END"]), row["Hit"], row["DEGRON"],row["E3"]])) 

        nouid_degron = list(d_nouid_degron_pos.keys())
        for index, uid in enumerate(nouid_degron):
            headers = d_nouid_degron_pos[uid]
            seq_local = ID_sequences[uid]
            seq_len = len(seq_local)
            seq_local = seq_local.upper()
            seq_local_list = np.array(list(seq_local))

            X_embedding = elmo_embedder.elmo_embedding(seq_local_list)
            protein_pad_global = np.zeros((seq_len + (shift * 2), 1024), dtype=np.float32)

            for i in range(0, seq_len, 1):
                protein_pad_global[i + (shift)] = X_embedding[i]

            protein_pad_local = ["-"] * (seq_len + (shift * 2))
            for i in range(0, seq_len, 1):
                protein_pad_local[i + (shift)] = seq_local[i]
            protein_pad_local = "".join(protein_pad_local)

            X_local, all_seq_transformed, all_seq_elmo_embedding = [], [], []
            for head in headers:
                head_arr = head.split("_")
                start_origin = int(head_arr[0])-1
                stop_origin = int(head_arr[1])
                motif = head_arr[2]
                degron = head_arr[3]
                E3_d = head_arr[-1]
                start = int(head_arr[0]) -1 + shift
                stop = int(head_arr[1]) + shift
                median_pos = (start+stop-1)//2
                slice_start = median_pos - slicesize // 2
                slice_stop = slice_start + slicesize
                query_seq = protein_pad_local[slice_start:slice_stop]
                seq_transformed = AA_encoding([query_seq])
                all_seq_transformed.append(seq_transformed)
                seq_elmo_embedding = protein_pad_global[slice_start:slice_stop]
                all_seq_elmo_embedding.append(seq_elmo_embedding)
                X_local.append([uid, E3_d, motif, degron, start_origin+1, stop_origin])

            probs_ = model_DL.predict([all_seq_elmo_embedding, all_seq_transformed]) 
            df_matches_seq_f = pd.DataFrame(X_local,columns=["Entry","E3","Hit","DEGRON","START","END"])             
            df_matches_seq_f['Prob_DEGRON'] = probs_

            all_data = pd.concat([all_data, df_matches_seq_f])
        
    tmp_folder = out_file # Please change this path to save result

    all_data1 = all_data[["Entry","E3", "Hit","DEGRON","START","END","Prob_DEGRON"]]
    all_data1['Prob_DEGRON'] =  all_data1['Prob_DEGRON'].astype(float)
    # all_data1['Prob_DEGRON'] = all_data1['Prob_DEGRON'].apply(lambda x: format(x, '.4f'))
    all_data2 = all_data1.sort_values(by=['Entry','Prob_DEGRON'],ascending=[True, False])
    
    re_names = {'E3': 'E3 ligase', 'Hit': 'Degron instance', 'DEGRON': 'Degron type',
            'START': 'Start', 'END': 'End', 'Prob_DEGRON': 'Score'}
    all_data2.rename(columns=re_names, inplace=True)    
    all_data2 = all_data2.round(4)

    path_output_dataframe = tmp_folder + '/prediction.txt'
    all_data2.to_csv(path_output_dataframe,sep="\t",index=False)

    print('Predictions completed, please check the results...')

In [10]:
degron_motif = ['CDC20','FBXW7']
fasta_file = './data/test.fasta'
out_file = './prediction'

pred_and_write_result_data(fasta_file, out_file, degron_motif)

Deep learning models are making predictions...
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


2024-05-12 22:02:25.592504: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/gridview//pbs/dispatcher/lib:/opt/gridview//pbs/dispatcher/lib::/usr/local/lib64:/usr/local/lib:/usr/local/lib64:/usr/local/lib
2024-05-12 22:02:25.592541: E tensorflow/stream_executor/cuda/cuda_driver.cc:318] failed call to cuInit: UNKNOWN ERROR (303)
2024-05-12 22:02:25.592562: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (node4): /proc/driver/nvidia/version does not exist
2024-05-12 22:02:25.593086: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 AVX512F FMA
2024-05-12 22:02:25.622346: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 2400000000 Hz
2024-05-12 


Predictions completed, please check the results...


### Predicting E3 ligase targeted degrons using XGBoost model trained on multimodal feature 

#### Specifically, the determination of residue-specific flexibility utilized the DynaMine software. Residue-specific solvent accessibility and secondary structures, including coiled coil and α-helix, were computed using the Spider2 tool. Protein disorder was assessed through the utilization of the IUPred software. The anchoring score of each degron was evaluated employing the ANCHOR program. Multiple sequence alignments (MSA) of orthologous proteins were acquired utilizing the Gopher tool from Bioware to calculate sequence conservation. Information pertaining to protein domains was retrieved from the Pfam database. Moreover, we evaluated the enrichment of important PTMs (phosphorylation and ubiquitination) within and around degrons. The experimentally verified PTMs information was downloaded from our constructed EPSD and PLMD resources.

In [23]:
out_file = './prediction'

test_data = './data/test_properties.tsv'
df_properties = pd.read_csv(test_data, sep="\t")

covariates = ['solvent_accessibility', 'disorder', 'conservative_score', 'coiled_coil',
              'anchoring_score', 'flanking_ptms', 'flanking_ub_lysine', 
              'a_helix', 'flexibility',  'structured_domain',  ]

clf = pickle.load(open(classifier_XGBoost, "rb"))
p_probs = clf.predict_proba(df_properties[covariates])
df_properties["degron_prob"] = [l[1] for l in p_probs]

path_output_dataframe = out_file + '/prediction.txt'
df_properties.to_csv(path_output_dataframe,sep="\t",index=False)

print('Predictions completed, please check the results...')


Predictions completed, please check the results...
